In [79]:
import torch
import pandas as pd
import os

In [80]:
path = "/Users/emilygong/Desktop/RUW-Oil-Prediction/data"
os.chdir(path)

In [116]:
guardian = pd.read_csv("Guardian_articles.csv")
guardian = guardian[['webPublicationDate', 'webTitle']]
nyt = pd.read_csv("NYT_articles.csv")
nyt = nyt[['headline', 'pub_date']]
# nyt.iat[19,0]
guardian

,webPublicationDate,webTitle
0,2022-10-22T01:24:17Z,Russian and US defence ministers discuss Ukrai...
1,2022-10-10T16:07:42Z,Russian pop star who criticised Ukraine war sa...
2,2022-08-26T17:50:05Z,Russia-Ukraine war: EU to hold urgent talks ov...
3,2022-10-15T18:12:08Z,Russia-Ukraine war live: ‘endemic corruption a...
4,2022-09-05T18:00:05Z,Russia-Ukraine war: Ukraine says it has pushed...
...,...,...
7155,2022-05-18T20:10:06Z,White House braced for North Korean nuclear te...
7156,2022-09-01T16:03:27Z,More rail strikes announced for September afte...
7157,2022-05-25T20:09:37Z,Georgia primaries deliver blow to Trump’s grip...
7158,2022-06-06T08:07:57Z,Island in the energy price storm: renewables h...


In [90]:
def clean(string):
    main = string.find("'main':")
    comma = string.find("',")
    return string[main+9:comma]

In [91]:
nyt['headline'] = nyt['headline'].apply(clean)

In [95]:
ha = nyt.iat[60,0]
ha

'Erdogan says Russia should return all captured Ukrainian territory.'

In [108]:
from transformers import pipeline

classifier = pipeline("text-generation", model = "gpt2", pad_token_id = 50256)
output = classifier(ha, max_new_tokens = 30)[0]['generated_text']
output


'Erdogan says Russia should return all captured Ukrainian territory. He said on Friday the Russian military carried out an operation that killed dozens of armed men and wounded five others.\n\nRussian military spokesman Igor Konas'

In [112]:

original = sentiment(ha)
# new = sentiment(output)
# print(original)
# print(new)
original[0]['label']
original[0]['score']

{'label': 'NEGATIVE', 'score': 0.9315783977508545}

In [161]:
sentiment = pipeline("sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")
classifier = pipeline("text-classification", model='bhadresh-savani/distilbert-base-uncased-emotion', top_k=None)

In [163]:
# guardian['sentiment'] = guardian['webTitle'].apply(sentiment)
titles = guardian['webTitle'].to_list()

label = []
score = []

anger = []
fear = []
joy = []
sadness = []
love = []
surprise = []

for title in titles:
    result = sentiment(title)
    label.append(result[0]['label'])
    score.append(result[0]['score'])
for title in titles:
    result = classifier(title)
    anger.append(result[0][0]['score'])
    fear.append(result[0][1]['score'])
    joy.append(result[0][2]['score'])
    sadness.append(result[0][3]['score'])
    love.append(result[0][4]['score'])
    surprise.append(result[0][5]['score'])

In [164]:
guardian['label'] = label
guardian['score'] = score

guardian['anger'] = anger
guardian['fear'] = fear
guardian['joy'] = joy
guardian['sadness'] = sadness
guardian['love'] = love
guardian['surprise'] = surprise

In [169]:
guardian.to_csv("guardian_sentiment.csv")

str